### Setup and Imports

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import config
from src import data_ingestion
from src.symbols import Symbols
from src.portfolio import Portfolio
from src.benchmark import Benchmark

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load Transaction Data

In [ ]:
# Create a list of all your transaction log files
log_files = [
    config.TRANS_LOG_DIR / "us_mkt_transactions.csv",
    config.TRANS_LOG_DIR / "exus_mkt_transactions.csv"
]

# Load and merge all found logs into a single DataFrame
master_log = data_ingestion.create_master_log(log_files).set_index("Date")
display(master_log)

### Initialize and Assess Symbols

In [ ]:
# 1. Initialize the Symbol manager
symbol_manager = Symbols(master_log)

# 2. Assess all symbols (checks cache and yfinance, creates templates for missing symbols)
symbol_manager.assess()

# 3. Display the results for user verification
print("✅ The following symbols were automatically identified:")
found_df = symbol_manager.get_found()
display(found_df)

print("❌ The following symbols were NOT found:")
missing_list = symbol_manager.get_missing()
print(missing_list)

### User Correction Step

Reviews the output from the cell above and lists any symbols that were incorrectly identified by the data provider (e.g. `yfinance`).

In [ ]:
# User reviews the table above and populates this list
incorrectly_identified_symbols = [
    'AMR', 'ASM'
]

# Mark these symbols as user-provided, which also updates the metadata template
symbol_manager.mark_as_manual(incorrectly_identified_symbols)

### Manual Task

1) Open the file: `data/manual-source/metadata.json`.
2) Fill in the `null` values for all the symbols listed.
3) For each of those same symbols, add a price history CSV file to the `data/manual-source/` directory.

### Reload Metadata and Get Final DataFrame

In [ ]:
# Reload the user-edited file from disk
symbol_manager.reload_user_metadata()

# Get the final, combined DataFrame of all symbol metadata
symbol_df = symbol_manager.get_unified_df()

print("--- Final Unified Symbol Metadata ---")
display(symbol_df)

### Initialize and Run Portfolio Analysis

In [ ]:
# Get the project date range
start_date, end_date, date_range, last_market_day = config.project_dates(
    master_log.index
)

# 1. Initialize the Portfolio analysis engine
portfolio = Portfolio(master_log, symbol_manager, date_range, last_market_day)

# 2. Run all calculations (fetching prices, calculating holdings and value)
portfolio.calculate_holdings_and_value()

### Display Results

In [ ]:
portfolio.get_current_holdings()

### Initialize and Run the Benchmark Simulation

In [ ]:
# Create an instance of the Benchmark class
benchmark_simulation = Benchmark(master_log, date_range, last_market_day)

# Run benchmark simulation
benchmark_simulation.run_simulation()

### Display Result

In [ ]:
# Now, you can get the populated DataFrame
benchmark_results_df = benchmark_simulation.get_results()

# Display the tail of the results to verify it worked
benchmark_results_df